In [2]:
from tqdm import tqdm
from pathlib import Path
import pandas as pd

def data(dataset_path):
    images = []
    labels = []
    dataset_path = Path(dataset_path)
    
    for subfolder in tqdm(dataset_path.iterdir(), desc="Processing Subfolders"):
        if subfolder.is_dir():
            for image_file in subfolder.iterdir():
                if image_file.is_file():
                    images.append(str(image_file))
                    labels.append(subfolder.name)
    
    df = pd.DataFrame({'image': images, 'label': labels})
    return df

segmented
grayscale
plantvillage dataset
color


In [3]:
segmented_path = '/kaggle/input/plantvillage-dataset/segmented'
segmented = data(segmented_path)
print(train_segmented.shape)
segmented.label.value_counts().to_frame()
num_classes = segmented['label'].nunique()
print(f"Number of classes: {num_classes}")

Processing Subfolders: 38it [01:45,  2.78s/it]

(54306, 2)
Number of classes: 38


In [6]:
train_segmented['label'].value_counts()

label
Orange___Haunglongbing_(Citrus_greening)              5507
Tomato___Tomato_Yellow_Leaf_Curl_Virus                5357
Soybean___healthy                                     5090
Peach___Bacterial_spot                                2297
Tomato___Bacterial_spot                               2127
Tomato___Late_blight                                  1909
Squash___Powdery_mildew                               1835
Tomato___Septoria_leaf_spot                           1771
Tomato___Spider_mites Two-spotted_spider_mite         1676
Apple___healthy                                       1645
Tomato___healthy                                      1591
Blueberry___healthy                                   1502
Pepper,_bell___healthy                                1478
Tomato___Target_Spot                                  1404
Grape___Esca_(Black_Measles)                          1384
Corn_(maize)___Common_rust_                           1192
Grape___Black_rot                                 

In [46]:
df = train_segmented
unique_labels = sorted(df["label"].unique())
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
df["label"] = df["label"].map(label_mapping)


In [8]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [58]:

import torch
from torch import Tensor
from torch import nn

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(6*6*256, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [59]:
import torch
import torch.nn as nn
from torchinfo import summary
model = AlexNet(num_classes=38)

summary(model, input_size=(1, 3, 224, 224))  # Batch size of 1, 3 color channels, 224x224 image


Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 38]                   --
├─Sequential: 1-1                        [1, 96, 26, 26]           --
│    └─Conv2d: 2-1                       [1, 96, 54, 54]           34,944
│    └─BatchNorm2d: 2-2                  [1, 96, 54, 54]           192
│    └─ReLU: 2-3                         [1, 96, 54, 54]           --
│    └─MaxPool2d: 2-4                    [1, 96, 26, 26]           --
├─Sequential: 1-2                        [1, 256, 12, 12]          --
│    └─Conv2d: 2-5                       [1, 256, 26, 26]          614,656
│    └─BatchNorm2d: 2-6                  [1, 256, 26, 26]          512
│    └─ReLU: 2-7                         [1, 256, 26, 26]          --
│    └─MaxPool2d: 2-8                    [1, 256, 12, 12]          --
├─Sequential: 1-3                        [1, 384, 12, 12]          --
│    └─Conv2d: 2-9                       [1, 384, 12, 12]          885,120

In [48]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df["label"], random_state=42)

class Dataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        
        image = Image.open(img_path).convert("RGB") 
        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

train_dataset = Dataset(train_df, transform=transform)
val_dataset = Dataset(val_df, transform=transform)
test_dataset = Dataset(test_df, transform=transform)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [50]:
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm


class Trainer:
    def __init__(self, model, train_loader, val_loader, test_loader, lr=1e-3, num_epochs=10):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.num_epochs = num_epochs
        self.lr = lr

        self.optimizer = optim.Adam(model.parameters(), lr=self.lr)
        self.criterion = nn.CrossEntropyLoss()

        self.best_val_loss = float("inf")
        self.best_model_state = None

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0

        pbar = tqdm(self.train_loader, desc=f"Epoch {epoch + 1}/{self.num_epochs}")

        for images, labels in pbar:
            images = images.float().to(self.device)
            labels = labels.long().to(self.device)  # CrossEntropyLoss expects long dtype

            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()
            pbar.set_postfix(loss=f"{loss.item():.4f}")

        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for images, labels in self.val_loader:
                images = images.float().to(self.device)
                labels = labels.long().to(self.device)

                outputs = self.model(images)
                loss = self.criterion(outputs, labels)

                total_loss += loss.item() * labels.size(0)

                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())

        avg_loss = total_loss / len(self.val_loader.dataset)
        metrics = self.calculate_metrics(all_preds, all_labels)

        return avg_loss, metrics

    @staticmethod
    def calculate_metrics(predictions, labels):
        accuracy = accuracy_score(labels, predictions)
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, predictions, average="weighted", zero_division=0
        )

        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

    @staticmethod
    def print_metrics(metrics, phase):
        print(f"\n{phase} Metrics:")
        print("-" * 50)
        for metric, value in metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        print("-" * 50)

    def train(self):
        for epoch in range(self.num_epochs):
            train_loss = self.train_epoch(epoch)
            val_loss, val_metrics = self.validate()

            print(f"\nEpoch {epoch + 1}: Train Loss = {train_loss:.4f} | Val Loss = {val_loss:.4f}")
            self.print_metrics(val_metrics, "Validation")

            # Save best model
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.best_model_state = self.model.state_dict()

    def test(self):
        self.model.load_state_dict(self.best_model_state)
        test_loss, test_metrics = self.validate()
        print("\nBest Model Performance on Test Set:")
        self.print_metrics(test_metrics, "Test")


In [61]:
trainer = Trainer(model, train_loader, test_loader, val_loader)

In [62]:
trainer.train()

Epoch 1/10: 100%|██████████| 1222/1222 [03:34<00:00,  5.70it/s, loss=1.8286]



Epoch 1: Train Loss = 2.5599 | Val Loss = 1.7110

Validation Metrics:
--------------------------------------------------
Accuracy: 0.4911
Precision: 0.4300
Recall: 0.4911
F1: 0.4124
--------------------------------------------------


Epoch 2/10: 100%|██████████| 1222/1222 [01:39<00:00, 12.34it/s, loss=1.2560]



Epoch 2: Train Loss = 1.6464 | Val Loss = 1.2823

Validation Metrics:
--------------------------------------------------
Accuracy: 0.6309
Precision: 0.6636
Recall: 0.6309
F1: 0.6062
--------------------------------------------------


Epoch 3/10: 100%|██████████| 1222/1222 [01:37<00:00, 12.48it/s, loss=0.8803]


...


Epoch 8: Train Loss = 0.3731 | Val Loss = 0.3666

Validation Metrics:
--------------------------------------------------
Accuracy: 0.8925
Precision: 0.8998
Recall: 0.8925
F1: 0.8903
--------------------------------------------------


Epoch 9/10: 100%|██████████| 1222/1222 [01:39<00:00, 12.30it/s, loss=0.2191]



Epoch 9: Train Loss = 0.3140 | Val Loss = 0.2765

Validation Metrics:
--------------------------------------------------
Accuracy: 0.9266
Precision: 0.9299
Recall: 0.9266
F1: 0.9269
--------------------------------------------------


Epoch 10/10:  92%|█████████▏| 1123/1222 [01:34<00:07, 12.65it/s, loss=0.2141]


Epoch 10: Train Loss = 0.2889 | Val Loss = 0.2427

Validation Metrics:
--------------------------------------------------
Accuracy: 0.9283
Precision: 0.9319
Recall: 0.9283
F1: 0.9285
--------------------------------------------------


In [64]:
torch.save(trainer.model.state_dict, 'model.pth')